# TP reconnaissance d'images

In [1]:
# img
import cv2
from skimage import io
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# files
import glob
from os import mkdir

In [3]:
# miscellaneous
import math
from time import time, localtime, strftime as time_tuple_to_str, gmtime as time_float_to_tuple
from tqdm import tqdm
from threading import Thread

## Retour sur SIFT

On va commencer avec une seule image:
    

In [4]:
# Adaptez la variable 'basedir' à votre cas
basedir= r'''C:\Users\stand\Desktop\Homework\Polytech - Informatique\Cours 2020-2021\S6\_8 Signals, Sounds and Images for the Informatician\2021.04.16 - TP #10 - Catégorisations d'images\raw-img''' + '\\'

In [5]:
################################################################
  ######## ################ EXAMPLES ################ ########
################################################################

In [6]:
# #img = io.imread(basedir+"train/farfalla/OIP-Zq_q8o8cw50F1x6S7AmlsQHaFi.jpeg")
# #img = io.imread(basedir+"train/farfalla/eb35b30729f1073ed1584d05fb1d4e9fe777ead218ac104497f5c97faeebb5bb_640.png")
# img = io.imread(basedir+"train/ragno/eb3cb3082af0073ed1584d05fb1d4e9fe777ead218ac104497f5c97ca5edb3bd_640.png")
# plt.imshow(img)
# print(img.shape)

In [7]:
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# sift = cv2.SIFT_create()
# now = time()
# kp,des = sift.detectAndCompute(gray,None)
# duration = time() - now
# print("durée = %.2f s" %duration)
# cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# plt.imshow(gray, cmap='gray')

In [8]:
################################################################
######## ################ EXAMPLES END ################ ########
################################################################

In [34]:
def build_SIFTs(listImages = [], n = 20, debug = True, SIFT_nbr_per_img = []):
    sifts = np.empty(shape=(0, 128), dtype=int)
    if n > 0:
        img_list = listImages[:int(n)]
    elif n == -1:
        img_list = listImages[:]
    else:
        img_list = listImages[:20]


    for pathImg in (tqdm(img_list, position=0, leave=True) if debug else img_list):
        img = io.imread(pathImg)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        # kp, des = sift.detectAndCompute(gray,None)
        des = sift.detectAndCompute(gray,None)[1]

        sifts = np.append(sifts, des, axis=0)
        
        SIFT_nbr_per_img.append(len(des))

    return sifts


def build_BoF(img_SIFTs, km):
    clusters_found = list(km.predict(img_SIFTs))
    return (1/len(img_SIFTs))*np.array([clusters_found.count(i) for i in range(len(km.cluster_centers_))])


def build_all_BoF(sifts, SIFT_nbr_per_img, km):
    
    result = []
    
    previous_sift_nbr = 0
    for sift_nbr_for_img in SIFT_nbr_per_img:
        result.append(build_BoF(sifts[previous_sift_nbr: previous_sift_nbr + sift_nbr_for_img], km))
        
        previous += sift_nbr_for_img
    
    return result


def less_quick_build_all_BoF(classes, basedir, km, n):
    ''' Equivalent, but much slower'''

    images = [img for cl in classes for img in glob.glob(basedir + cl + '/*')[:n]]
    
    k = len(km.cluster_centers_)

    SIFT_nbr_per_img = [] # To transform the BoF of each image from int value (count) to percentage

    all_SIFTs = build_SIFTs(images, n = -1, debug = False, SIFT_nbr_per_img = SIFT_nbr_per_img)
    
    clusters_found = list(km.predict(all_SIFTs))
    
    
    result = np.empty(shape=(0, k), dtype=float)
    
    previous_SIFTs_nbr = 0
    for img_i in range(len(images)):
        actual_SIFTs_nbr = SIFT_nbr_per_img[img_i]

        this_img_BoF = np.array([clusters_found[previous_SIFTs_nbr : previous_SIFTs_nbr + actual_SIFTs_nbr].count(i)\
                                         for i in range(k)], dtype = float) # BoF
        this_img_BoF /= actual_SIFTs_nbr # fraction

        result = np.append(result, [this_img_BoF], axis = 0)
        previous_SIFTs_nbr += actual_SIFTs_nbr
            
    return result

In [10]:
def time_str():
    return time_tuple_to_str('''%Y.%m.%d %Hh%Mm%Ss''', localtime())

In [11]:
# Methods to save data

pickle_folder = 'pickle_saves\\'
pickle_extension = '.pkl'

def make_pkl_dir():
    global pickle_folder
    try:
        mkdir(basedir + pickle_folder)
    except FileExistsError:
        pass

def save(array, name = None):
    global basedir, pickle_folder, pickle_extension

    if name == None:
        name = time_str() + ' - array_saved' + pickle_extension
    elif not name.endswith(pickle_extension):
        name += pickle_extension
    
    make_pkl_dir()
    
    array.dump(basedir + pickle_folder + name)
    return name

def load(name):
    global basedir, pickle_folder

    file_name = basedir + pickle_folder + name
    
    make_pkl_dir()
    try:
        array = np.load(file_name, allow_pickle = True)
    except:
        print(f'Somthing went wrong while trying to retrieve data from "{file_name}"')
        return np.array([])
    else:
        return array

Que contiennent les variables kp et des ? Quelles sont les dimensions ?

In [12]:
#print('len(kp):', len(kp))
#print('type(kp[0]):', type(kp[0]))
#print()
#print('des[0]:', des[0])
#print('type(des[0]):', type(des[0]))
#print('len(des[0]):', len(des[0]))
#print()
#print('des:', des)
#print('type(des):', type(des))
#print('len(des):', len(des))
#print('des.shape:', des.shape)

## Chargement de vos classes d'images

In [13]:
baseTrain = basedir + 'train/'
baseTest = basedir + 'test/'
classes = ['pecora','ragno']

In [37]:
nb = {} # nombre d'images par classe
SIFTs = {} # SIFTs par classe

SIFT_nbr_per_img_per_cl = {}

n = 20 # échantillon d'images

for cl in classes:
    SIFT_nbr_per_img_per_cl[cl] = []
    
    prev = time()
    listImages = glob.glob(baseTrain + cl + '/*')
    print('class:', cl, ':', len(listImages), 'images.') 
    nb[cl] = len(listImages)
    SIFTs[cl] = build_SIFTs(listImages, n = n, SIFT_nbr_per_img = SIFT_nbr_per_img_per_cl[cl])

    print('classe {}: {:.2f} s\n'.format(cl, time() - prev))

print('total number of images:', sum(nb.values()))
print('longueur des SIFTs')
print(*(f'{cl}: ' + str(len(SIFTs[cl])) for cl in classes), sep = ', ')
print(SIFT_nbr_per_img_per_cl)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

class: pecora : 1456 images.


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

classe pecora: 3.71 s

class: ragno : 3856 images.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.20it/s]

classe ragno: 3.96 s

total number of images: 5312
longueur des SIFTs
pecora: 36249, ragno: 31754
{'pecora': [3448, 4638, 6607, 7333, 7736, 9165, 11526, 13416, 14764, 16620, 18788, 20927, 22824, 24266, 26570, 27107, 28792, 30396, 32608, 36249], 'ragno': [356, 2932, 3735, 4906, 5082, 6108, 8073, 10689, 14066, 17279, 19470, 20474, 21319, 25400, 27233, 28066, 29199, 30441, 30868, 31754]}


In [15]:
for cl in classes:
    print(cl + ':', SIFTs[cl].shape)

pecora: (36249, 128)
ragno: (31754, 128)


## Sac de descripteurs (BoF)

In [16]:
k_min = 2
k_max = 50

In [17]:
from sklearn.cluster import MiniBatchKMeans#, KMeans

In [18]:
# merge sifts from each class in one list
all_SIFTs = np.empty(shape=(0, 128), dtype=int)
for cl in classes:
    all_SIFTs = np.append(all_SIFTs, SIFTs[cl], axis = 0)

In [19]:
################
### Single k ###
################

In [20]:
k = 50

K_Means = MiniBatchKMeans(n_clusters = k).fit(all_SIFTs)
labels = K_Means.labels_ # for each SIFT, a cluster label

# print({i: list(labels).count(i) for i in range(k)})


#KMeans_dic = {} # KMeans par classe
#labels = {} # labels par classe
#for cl in classes:
#    KMeans_dic[cl] = MiniBatchKMeans(n_clusters = k).fit(SIFTs[cl])
#    labels[cl] = KMeans_dic[cl].labels_ # for each SIFT, a cluster label
#    print(labels[cl])
#    print({i: list(labels[cl]).count(i) for i in range(k)})

In [21]:
prev = time()

BoFs_both_classes = build_all_BoF(classes, baseTrain, K_Means, n)

print('known lenght:', len(BoFs_both_classes[0]))
print(BoFs_both_classes[0])

print(f'{time() - prev}s elapsed')

known lenght: 50
[0.03422274 0.02639211 0.01769142 0.01421114 0.01450116 0.02552204
 0.03016241 0.02523202 0.00435035 0.01363109 0.01682135 0.02349188
 0.02668213 0.01508121 0.02813225 0.02204176 0.02552204 0.00696056
 0.02204176 0.01479118 0.02175174 0.01972158 0.02175174 0.02204176
 0.01508121 0.01740139 0.01740139 0.01653132 0.02204176 0.02668213
 0.02233179 0.02407193 0.01595128 0.01653132 0.02204176 0.02146172
 0.01885151 0.01537123 0.02697216 0.01450116 0.03364269 0.01537123
 0.01682135 0.02581206 0.01537123 0.02059165 0.02059165 0.01769142
 0.01740139 0.01073086]
13.841761589050293s elapsed


In [22]:
prev = time()

BoFs_both_classes = less_quick_build_all_BoF(classes, baseTrain, K_Means, n)

print('known lenght:', len(BoFs_both_classes[0]))
print(BoFs_both_classes[0])

print(f'{time() - prev}s elapsed')

known lenght: 50
[0.03422274 0.02639211 0.01769142 0.01421114 0.01450116 0.02552204
 0.03016241 0.02523202 0.00435035 0.01363109 0.01682135 0.02349188
 0.02668213 0.01508121 0.02813225 0.02204176 0.02552204 0.00696056
 0.02204176 0.01479118 0.02175174 0.01972158 0.02175174 0.02204176
 0.01508121 0.01740139 0.01740139 0.01653132 0.02204176 0.02668213
 0.02233179 0.02407193 0.01595128 0.01653132 0.02204176 0.02146172
 0.01885151 0.01537123 0.02697216 0.01450116 0.03364269 0.01537123
 0.01682135 0.02581206 0.01537123 0.02059165 0.02059165 0.01769142
 0.01740139 0.01073086]
16.21074151992798s elapsed


In [23]:
################
# Single k end #
################

In [24]:
################
#### Many k ####
################

In [27]:
print('K_Means')
K_means_dico = {}

prev = time()

for k in range(k_min, k_max):
    K_means_dico[k] = MiniBatchKMeans(n_clusters = k).fit(all_SIFTs)
    
    print(f'{k+1}/100 in {int(time() - prev)}s', end = '\r')

K_Means


In [106]:
print('BoFs')
BoFs_both_classes_dico = {}

prev = time()

for k in range(k_min, k_max):
    BoFs_both_classes_dico[k] = build_all_BoF(classes, baseTrain, K_means_dico[k], n)

    print(f'{k+1}/100 in {int(time() - prev)}s', end = '\r')

BoFs


## Classification par regression logistique

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [94]:
#log_reg = LogisticRegression()

#groundTruth = [0]*n + [1]*n
#log_reg.fit(sample_BoFs_both_classes, groundTruth)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
#resTrain = log_reg.predict(sample_BoFs_both_classes)
#scoreTrain = f1_score(groundTruth, resTrain)
#print(f'train score pour k = {k}:', scoreTrain)

train score pour k = 50: 0.6


In [120]:
print('scores train')
scoresTrain_dico = {}
log_reg_dico = {}

groundTruth = [0]*n + [1]*n

for k in range(k_min, k_max):
    log_reg = LogisticRegression()
    log_reg.fit(BoFs_both_classes_dico[k], list(groundTruth))

    scoresTrain_dico[k] = f1_score(list(groundTruth), log_reg.predict(BoFs_both_classes_dico[k]))

    log_reg_dico[k] = log_reg
    
print(scoresTrain_dico)

scores train
{2: 0.6, 3: 0.6, 4: 0.6, 5: 0.6, 6: 0.6, 7: 0.6, 8: 0.6, 9: 0.6, 10: 0.6, 11: 0.6, 12: 0.6, 13: 0.6, 14: 0.6, 15: 0.6, 16: 0.6, 17: 0.6, 18: 0.6, 19: 0.6, 20: 0.6, 21: 0.6, 22: 0.6, 23: 0.6, 24: 0.6, 25: 0.6, 26: 0.6, 27: 0.6, 28: 0.6, 29: 0.6, 30: 0.6, 31: 0.6, 32: 0.6, 33: 0.6, 34: 0.6, 35: 0.6, 36: 0.6, 37: 0.6, 38: 0.6, 39: 0.6, 40: 0.6, 41: 0.6, 42: 0.6, 43: 0.6, 44: 0.6, 45: 0.6, 46: 0.6, 47: 0.6, 48: 0.6, 49: 0.6}


In [123]:
best_k = max(scoresTrain_dico.values())
for k in scoresTrain_dico:
    if scoresTrain_dico[k] == best_k:
        best_k = k
        break
else:
    raise ValueError('k were not found, it is a problem')

print(best_k)

2


## test dataset

In [125]:
for cl in classes:
    print(f'test images from class "{cl}"')
    for img in glob.glob(baseTest + cl + '/*')[:n]:
        
        bof_test = build_BoF(img, K_means_dico[best_k])
        
        if f1_score(groundTruth, log_reg_dico[k].predict([bof_test])) >= 0.5:
            print('class found:', cl)
        else:
            print(f'class not found (not {cl})')

classe "pecora"
[array([0.38040557, 0.61959443])]


ValueError: X has 2 features per sample; expecting 50